# Azure AI Search LangChain vector code sample
This code demonstrates how to use Azure AI Search with OpenAI and the `langchain.vectorstores.azuresearch`` module.
To run the code, install the following packages, including the `azure-search-documents==11.4.0b8` packaged, as noted in [LangChain docs](https://python.langchain.com/docs/integrations/vectorstores/azuresearch).

In [ ]:
# ! pip install azure-search-documents==11.4.0b8 
# ! pip install openai
# ! pip install python-dotenv
# ! pip install langchain

# Langchain functions!


https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/vectorstores/azuresearch.py

## Import required libraries and environment variables

In [1]:
# Import required libraries  
import openai
import os  
from dotenv import load_dotenv
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from azure.search.documents.indexes.models import (
    SemanticSettings,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField
)


## Configure Azure OpenAI settings

In [3]:
# Configure environment variables 
import os
from azure.core.credentials import AzureKeyCredential
os.environ["AZURE_OPENAI_API_KEY"]  = 'b82effcf491e45a088b1cd578713311c'
os.environ["OPENAI_API_VERSION"]    = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://usesharedaopenai001.openai.azure.com/'
os.environ["OPENAI_API_TYPE"]       = 'azure'
# Variables-------------------------------------------------------
index                               = "azure-cognitive-search-vector-demo"
azure_search_endpoint               = 'https://genai0.search.windows.net'
MODEL                               = "gtp35turbo-latest"
key                                 = 'lvhCA67EeE3JRyxyem5L0wGJSfOxscm2jft887ECdJAzSeDzoCNZ'
model                               = "text-embedding-ada-002"
credential                          = AzureKeyCredential(key)
COMPLETION_TOKENS                   = 1000
top_search_vector_k                 = 5

openai.api_type: str = os.getenv("OPENAI_API_TYPE")  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("OPENAI_API_VERSION")  

# https://github.com/langchain-ai/langchain/issues/9765
os.environ["AZURESEARCH_FIELDS_ID"] = "id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "page_content"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "contentVector"
os.environ["AZURESEARCH_FIELDS_METADATA"] = "source"


vector_store_address = azure_search_endpoint
vector_store_password = key
index_name = index

## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your search index:

In [4]:
embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        chunk_size=1000)

In [5]:
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name='config',
        semantic_settings=SemanticSettings(
            default_configuration='config',
            configurations=[
                SemanticConfiguration(
                    name='config',
                    prioritized_fields=PrioritizedFields(
                        title_field=SemanticField(field_name='company_name'),
                        prioritized_content_fields=[SemanticField(field_name='source')],
                        prioritized_keywords_fields=[SemanticField(field_name='doc_type')]
                    ))
            ])
    )


Subtype value exhaustiveKnn has no mapping, use base class VectorSearchAlgorithmConfiguration.


## Perform a vector similarity search

In [6]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What is the Revenue of Microsoft",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

HttpResponseError: (InvalidRequestParameter) Unknown field 'content_vector' in vector field list.
Code: InvalidRequestParameter
Message: Unknown field 'content_vector' in vector field list.
Exception Details:	(UnknownField) Unknown field 'content_vector' in vector field list.
	Code: UnknownField
	Message: Unknown field 'content_vector' in vector field list.

## Perform a hybrid search

In [18]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3, 
    search_type="hybrid"
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Perform a hybrid search with semantic reranking (powered by Bing)

In [19]:
# Perform a hybrid search with semantic reranking  
docs_and_scores = vector_store.semantic_hybrid_search_with_score(  
    query="What did the president say about Ketanji Brown Jackson",  
    k=3,  
)  
  
# Print the results  
for doc, score in docs_and_scores:  
    print("-" * 80)  
    answers = doc.metadata['answers']  
    if answers:  
        if answers.get('highlights'):  
            print(f"Semantic Answer: {answers['highlights']}")  
        else:  
            print(f"Semantic Answer: {answers['text']}")  
        print(f"Semantic Answer Score: {score}")  
    print("Content:", doc.page_content)  
    captions = doc.metadata['captions']
    print(f"Score: {score}") 
    if captions:  
        if captions.get('highlights'):  
            print(f"Caption: {captions['highlights']}")  
        else:  
            print(f"Caption: {captions['text']}")  
    else:  
        print("Caption not available")  


--------------------------------------------------------------------------------
Content: Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
Score: 0.03333333507180214
Caption: One of the most serious constitutional responsibilities a President has is nominating 